In [ ]:
from DataHandling import ReadBinary, GetSBCEvent    
from DataHandling.ReadBinary import ReadBlock as rb
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
import numpy as np

from scipy import optimize
import re

In [ ]:
mpl.rc("font", size=14)

In [ ]:
ev = dict(acoustic=rb("/exp/e961/data/users/gputnam/test-daq/50kHz.sbc.bin"))
# ev = dict(acoustic=rb("/exp/e961/data/users/gputnam/test-daq/acoustic-191111-0802.sbc.bin"))
ev["acoustic"]["loaded"] = True

nsample = 0
PERIOD = 100e-9 # s
for k in list(ev["acoustic"].keys()):
    m = re.search("Channel(\d+)", k)
    if m:
        rnge = ev["acoustic"][k][0] / 1e3 # mV -> V
        offset = ev["acoustic"][k][1] / 1e3
        ev["acoustic"][k] = ev["acoustic"][k][2:]*rnge + offset
        nsample = ev["acoustic"][k].size
ev["acoustic"]["time"] = np.arange(nsample)*PERIOD

In [ ]:
def BandPass2(yd, f_low, f_high):
    fband = np.array([f_low, f_high])
    b, a = scipy.signal.butter(2, fband / (2.5e6 / 2.0), btype='bandpass', output='ba')
    yd_f = scipy.signal.filtfilt(b, a, yd)
    return yd_f

f_high=np.float64(40e3)
f_low=np.float64(6e3)
meansamp=np.intp(1e4)
bs_win=np.float64([-0.15, -0.12])
t0_win=np.float64([-0.12, 0])
led_tau=np.float64(2e-4)

In [ ]:
raw_piezo

In [ ]:
fastdaq_time = ev["acoustic"]["time"]
dt = fastdaq_time[1] - fastdaq_time[0]

In [ ]:
raw_piezo = ev["acoustic"]["Channel1"]
plt.plot(fastdaq_time*1e6, raw_piezo)
plt.xlim([0, 100])
plt.xlabel("Time [$\\mu$s]")
plt.ylabel("Amplitude [V]")
# plt.axvline(ana["bubble_t0"][event][0], color="red")

In [ ]:
PIEZO = 1
raw_piezo = ev["acoustic"]["Channel%i" % PIEZO] 

In [ ]:
dt

In [ ]:
f, t, Sxx = scipy.signal.spectrogram(raw_piezo, fs=1./dt, nfft=512, noverlap=450,
                                              mode="psd", window="hann", nperseg=512)

In [ ]:
from scipy.fft import fftshift
plt.pcolormesh(t + fastdaq_time.min(), fftshift(f)[fftshift(f) < 5e5]/1e3, 
               fftshift(Sxx, axes=0)[fftshift(f) < 5e5, :], shading='gouraud', vmin=0, vmax=1e-7)
plt.xlabel("Time [s]")
plt.ylabel("Frequency [kHz]")
cbar = plt.colorbar()
cbar.set_label('Spectogram Amplitude', rotation=270, labelpad=20)

# plt.title("Run Directory %s Event %i Piezo %i" % (rundir.split("/")[-1], event, PIEZO), pad=20)
# plt.axvline(ana["bubble_t0"][event][0], color="red")

# plt.savefig("figures/spectogram_R%s_E%i_P%i.png" % (rundir.split("/")[-1], event, PIEZO))

In [ ]:
t0_win_ix = np.intp(np.round((t0_win - fastdaq_time[0]) / dt))
bs_win_ix = np.intp(np.round((bs_win - fastdaq_time[0]) / dt))

try:
    led_on = ev["fastDAQ"]["CAMgate"] < -0.5
except:
    led_on = np.zeros(raw_piezo.shape)
    
led_switch = np.diff(np.int8(led_on))
led_switch_on = led_switch == 1
led_switch_off = led_switch == -1
led_switch_on_time = fastdaq_time[:-1][led_switch_on]
led_switch_off_time = fastdaq_time[:-1][led_switch_off]

filtered_piezo = BandPass2(raw_piezo - np.mean(raw_piezo[:meansamp]), f_low, f_high)
led_k = 1./led_tau
piezo_led_on = np.sum((fastdaq_time[:, None] > led_switch_on_time) *
                      np.exp(led_k * np.minimum(led_switch_on_time -
                                                fastdaq_time[:, None],
                                                np.float64([0]))),
                      axis=1)
piezo_led_off = np.sum((fastdaq_time[:, None] > led_switch_off_time) *
                       np.exp(led_k * np.minimum(led_switch_off_time -
                                                 fastdaq_time[:, None],
                                                 np.float64([0]))),
                       axis=1)

filtered_piezo_led_on = BandPass2(piezo_led_on, f_low, f_high)
filtered_piezo_led_off = BandPass2(piezo_led_off, f_low, f_high)
fit_func = lambda _, on_amp, off_amp: on_amp*filtered_piezo_led_on[tdata] + off_amp*filtered_piezo_led_off[tdata]
tdata = np.arange(0, bs_win_ix[1])
ydata = filtered_piezo[tdata]
x0_guess = np.array([1., -1.])
led_amp, conv = optimize.curve_fit(fit_func, tdata, ydata, x0_guess)

led_only = led_amp[0] * piezo_led_on + led_amp[1] * piezo_led_off
led_amp

In [ ]:
def freq_filter(freqs, lower=None, upper=None):
    # Inputs:
    #   freqs: An array of frequency bins
    #   lower: The lower frequency to cut-off at
    #   upper: The upper frequency to cut-off at
    # Outputs: An array of indices where the the frequency in freqs is between lower and upper
    if lower is None and upper is None:
        return freqs
    if lower is None:
        return np.where([x <= upper for x in freqs])
    if upper is None:
        return np.where([x >= lower for x in freqs])
    return np.where([lower <= x <= upper for x in freqs])


def spectrum_sums(spectrum, fr, n, lowerf=None, upperf=None):
    # Inputs:
    #   spectrum: The output 2d spectrum from a spectogram
    #   fr: A list of frequency bins corresponding to the spectrum
    #   n: Number of bins
    #   lowerf: The lower frequency to cut-off at
    #   upperf: The upper frequency to cut-off at
    # Outputs: A compressed 1d array where each element is the sum of a bin from spectrum, only counting
    #          frequencies between lowerf and upperf
    out = []
    good_indices = freq_filter(fr, lowerf, upperf)
    for subn in range(n):
        out.append(np.trapz(spectrum[good_indices[0], subn], dx=np.mean(np.diff(fr))))
    return out

def rescale_window(w1, w2):
    # Inputs:
    #   w1: An array with 2 elements
    #   w2: An array with 2 elements
    # Outputs: A rescaled version of w2 so tha the endpoints of w2 match w1 but the number of elements remain the same
    y1, y2 = min(w1), max(w1)
    x1, x2 = min(w2), max(w2)
    if x1 == x2:
        return 0*w2
    a = (y1-y2)/(x1-x2)
    b = (x1*y2-x2*y1)/(x1-x2)
    return a*w2+b

def corr_signal(tau, dt, t0, n, fit_type=0, shift=10):
    # Inputs:
    #   tau: Time constant on exponential decay
    #   dt: Step size for the x-axis
    #   t0: Where the exponential signal will start. Not important when used with correlation
    #   N: Number of points requested
    #   fit_type: The type of signal to create. See corr_signal_type_templates.py for a better explanation.
    #               fit_type = 0 --> Exponential decay
    #               fit_type = 1 --> Constant 1 followed by exponential decay (continuous)
    #               fit_type = 2 --> Linear increase followed by exponential decay
    #               fit_type = 3 --> Log increase followed by exponential decay
    #               fit_type = 4 --> 0 value followed by an exponential decrease. Discontinuous.
    # Outputs:
    #   t: t-values for plotting
    #   y: y-values of our filter signal.
    # After careful analysis, we've determined that there reaches a point in the filtered piezo signal that
    # exhibits a sharp increase followed by an exponential decay. This function returns a brief exponential
    # decay function for use with convolution/correlation.
    shift = int(np.ceil(shift))
    t = np.linspace(t0, t0+dt*n, n)
    y = np.exp(-(t-t0)/tau)
    ycopy = copy.deepcopy(y)
    if fit_type == 0:
        pass
    elif fit_type == 1:
        for subn in range(len(y) - shift):
            y[subn+shift] = ycopy[subn]
        y[0:shift] = 1
    elif fit_type == 2:
        for subn in range(len(y) - shift):
            y[subn + shift] = ycopy[subn]
        y[0:shift] = (t[0:shift] - t0)/(shift*dt)
    elif fit_type == 3:
        for subn in range(len(y) - shift):
            y[subn + shift] = ycopy[subn]
            y[0:shift] = np.log((t[0:shift] + 1 - t0)) / np.log(shift*dt + 1)
    elif fit_type == 4:
        for subn in range(len(y) - shift):
            y[subn+shift] = ycopy[subn]
            y[0:shift] = 0
    return t, y
def find_t0_from_corr(corrt, corry):
    # Inputs:
    #   corrt: Time-values of the correlation signal
    #   corry: Y-values of the correlation signal
    # Outputs: The time of the maximum in corry such that corrt is less than or equal to 0.
#     n = np.where(corrt >= 0)
#     corry[n] = 0
    return corrt[np.argmax(corry)]


import copy

In [ ]:
tau_peak = 0.0025884277467056165  # <-- This comes from TauResultAnalysis.py (J.G.)
tau_average = 0.0038163479219674467  # <-- This also ^^

# corr_lowerf = 20000
# corr_upperf = 40000

corr_lowerf = 1000
corr_upperf = 25000

piezo_fit_type = 0

tau = tau_average

In [ ]:
piezo_waveform = raw_piezo - led_only
piezo_timebase = fastdaq_time
lower = corr_lowerf
upper = corr_upperf

In [ ]:
timebase = piezo_timebase
textent = [min(timebase), max(timebase)]
dt = np.mean(np.diff(timebase))
fr, bn, sp = scipy.signal.spectrogram(piezo_waveform, fs=1./dt, nfft=512, noverlap=450,
                              mode="psd", window="hann", nperseg=512)
n = len(bn)
sp_sums = spectrum_sums(sp, fr, n, lower, upper)
sp_sums = scipy.signal.medfilt(sp_sums)
rescaled_t = rescale_window(textent, bn)
corr_dt = np.mean(np.diff(rescaled_t))
corr_n = 1000
corr_t, corr_y = corr_signal(tau, corr_dt, rescaled_t[0], corr_n, fit_type=piezo_fit_type)
corr = np.correlate(sp_sums, corr_y, "same")
corr_t = rescaled_t - 0.5 * corr_n * corr_dt
corr = np.correlate(sp_sums, corr_y, "same")
corr_t = rescaled_t - 0.5 * corr_n * corr_dt
test_t0 = find_t0_from_corr(corr_t, corr) # This is the t0 we begin to look backwards from

In [ ]:
plt.plot(corr_t, sp_sums)
plt.ylabel("Filtered Summed Spectogram Window")
plt.xlabel("Time [s]")
plt.title("Run Directory %s Event %i Piezo %i" % (rundir.split("/")[-1], event, PIEZO))

plt.tight_layout()
plt.savefig("figures/correlated_summed_spectogram_amplitude_R%s_E%i_P%i.png" % (rundir.split("/")[-1], event, PIEZO))

In [ ]:
plt.plot(corr_t, corr)
plt.ylabel("Exponential Decay Correlation")
plt.xlabel("Time [s]")
plt.title("Run Directory %s Event %i Piezo %i" % (rundir.split("/")[-1], event, PIEZO))


plt.axvline(test_t0, color="tab:orange", linestyle="--")
plt.text(t0-0.005, 0.8,  "Initial T0", 
         color="tab:orange", horizontalalignment="right")

plt.tight_layout()
plt.savefig("figures/median_correlated_summed_spectogram_amplitude_R%s_E%i_P%i.png" % (rundir.split("/")[-1], event, PIEZO))

In [ ]:
def my_rms(arr):
    #return np.sqrt(arr.dot(arr)/arr.size)
    return np.std(arr)

log_sp_sums = np.log(sp_sums)
first_on = np.argmax(led_on)
first_off = np.argmin(led_on[first_on:])
second_on = np.argmax(led_on[first_off:])
t_first_off = timebase[first_off]
t_second_on = timebase[second_on]
rescaled_t_first_off_index = np.argmin(np.abs(rescaled_t - t_first_off))
rescaled_t_second_on_index = np.argmin(np.abs(rescaled_t - t_second_on))

baseline = np.average(log_sp_sums[rescaled_t_first_off_index+1:rescaled_t_second_on_index])

baseline_rms = my_rms(log_sp_sums[rescaled_t_first_off_index+1:rescaled_t_second_on_index])


In [ ]:
test_t0_index = np.argmin(np.abs(rescaled_t - test_t0))
rescaled_dt = np.mean(np.diff(rescaled_t))
t_thresh = 100e-6

n_lookback = int(np.floor(t_thresh/rescaled_dt))
pts_lookbacked_sofar = 0


In [ ]:
while True:
    to_test = log_sp_sums[test_t0_index-n_lookback-pts_lookbacked_sofar:test_t0_index-pts_lookbacked_sofar]

    if np.all(to_test<(baseline+5*baseline_rms)):
        break
    pts_lookbacked_sofar += 1
    if test_t0_index-n_lookback-pts_lookbacked_sofar <= 0:
        pts_lookbacked_sofar = -1
        break
if pts_lookbacked_sofar != -1:
    t0 = rescaled_t[test_t0_index-pts_lookbacked_sofar] + rescaled_dt/2


In [ ]:
plt.plot(fastdaq_time, raw_piezo, label="Raw Piezo")
plt.plot(fastdaq_time, raw_piezo - led_only, label="LED Subtracted")
# plt.axvline(test_t0, color="red")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.title("Run Directory %s Event %i Piezo %i" % (rundir.split("/")[-1], event, PIEZO))
plt.legend()
plt.savefig("figures/waveform_R%s_E%i_P%i.png" % (rundir.split("/")[-1], event, PIEZO))

In [ ]:
plt.plot(rescaled_t, log_sp_sums)
plt.axhline(baseline, color="black")
plt.text(0.05, baseline-0.25, "Baseline", verticalalignment="top")

plt.axhline(baseline + 5*baseline_rms, color="red")
plt.text(-0.15, baseline + 5*baseline_rms + 0.25, "Baseline + 5$\\cdot$rms", color="red")

plt.axvline(test_t0, color="tab:orange", linestyle="--")
plt.axvline(t0, color="tab:orange")
plt.text(t0-0.005, plt.ylim()[1]-0.5,  "Initial &\nRefined T0", 
         color="tab:orange", horizontalalignment="right", verticalalignment="top")


plt.ylabel("log Summed Spectogram Window")
plt.xlabel("Time [s]")
plt.title("Run Directory %s Event %i Piezo %i" % (rundir.split("/")[-1], event, PIEZO))

plt.savefig("figures/log_summed_spectogram_R%s_E%i_P%i.png" % (rundir.split("/")[-1], event, PIEZO))